In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "COVID-19"
cohort = "GSE275334"

# Input paths
in_trait_dir = "../../input/GEO/COVID-19"
in_cohort_dir = "../../input/GEO/COVID-19/GSE275334"

# Output paths
out_data_file = "../../output/preprocess/COVID-19/GSE275334.csv"
out_gene_data_file = "../../output/preprocess/COVID-19/gene_data/GSE275334.csv"
out_clinical_data_file = "../../output/preprocess/COVID-19/clinical_data/GSE275334.csv"
json_path = "../../output/preprocess/COVID-19/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Immune Exhaustion in ME/CFS and long COVID"
!Series_summary	"Gene expression analysis of RNA was performed using the commercially available NanoString® nCounter Immune Exhaustion gene expression panel (NanoString Technologies, Seattle, WA, USA). This panel contains 785 genes to elucidate mechanisms behind T cell, B cell and NK cell exhaustion in disease."
!Series_summary	"Ribonucleic acid (RNA) was extracted from peripheral blood mononuclear cells (PBMCs) isolated from ME/CFS (n=14), long COVID (n=15), and healthy control (HC; n=18) participants. ME/CFS participants were included according to Canadian Consensus Criteria for ME. Long COVID participants were eligible according to the working case definition for Post COVID-19 Condition published by the World Health Organization."
!Series_overall_design	"Raw gene expression data was normalised against positive and negative controls to account for background noise and platform-associated variation. Nor

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to contain gene expression data
# from NanoString nCounter Immune Exhaustion gene expression panel, which includes 785 genes.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics dictionary:
# - trait (COVID-19) can be inferred from "disease" in row 3
# - age is available in row 1
# - gender/sex is available in row 2
trait_row = 3
age_row = 1
gender_row = 2

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> int:
    """
    Convert trait values to binary (0 or 1).
    For this dataset, we're looking for COVID-19 which maps to "Long COVID".
    """
    if value is None:
        return None
    
    # Extract the value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary based on whether it's "Long COVID" or not
    if "long covid" in value.lower():
        return 1
    else:
        return 0

def convert_age(value: str) -> float:
    """
    Convert age values to continuous (float).
    """
    if value is None:
        return None
    
    # Extract the value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value: str) -> int:
    """
    Convert gender values to binary (0 for female, 1 for male).
    """
    if value is None:
        return None
    
    # Extract the value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.lower() == "female":
        return 0
    elif value.lower() == "male":
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None
# Initial filtering on usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Using the clinical_data variable that should be available from previous steps
    try:
        # Extract clinical features using the pre-existing clinical_data DataFrame
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        preview = preview_df(selected_clinical_df)
        print("Clinical Data Preview:")
        print(preview)
        
        # Save to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical features: {e}")


Clinical Data Preview:
{'GSM8475033': [0.0, 24.0, 0.0], 'GSM8475034': [0.0, 46.0, 0.0], 'GSM8475035': [0.0, 50.0, 0.0], 'GSM8475036': [0.0, 37.0, 1.0], 'GSM8475037': [0.0, 19.0, 0.0], 'GSM8475038': [0.0, 40.0, 0.0], 'GSM8475039': [0.0, 46.0, 1.0], 'GSM8475040': [0.0, 63.0, 0.0], 'GSM8475041': [0.0, 54.0, 0.0], 'GSM8475042': [0.0, 46.0, 0.0], 'GSM8475043': [0.0, 48.0, 0.0], 'GSM8475044': [0.0, 34.0, 1.0], 'GSM8475045': [0.0, 22.0, 1.0], 'GSM8475046': [0.0, 59.0, 0.0], 'GSM8475047': [0.0, 39.0, 0.0], 'GSM8475048': [0.0, 27.0, 0.0], 'GSM8475049': [0.0, 61.0, 0.0], 'GSM8475050': [0.0, 38.0, 1.0], 'GSM8475051': [1.0, 44.0, 0.0], 'GSM8475052': [1.0, 41.0, 1.0], 'GSM8475053': [1.0, 49.0, 0.0], 'GSM8475054': [1.0, 19.0, 0.0], 'GSM8475055': [1.0, 38.0, 0.0], 'GSM8475056': [1.0, 43.0, 0.0], 'GSM8475057': [1.0, 62.0, 0.0], 'GSM8475058': [1.0, 30.0, 0.0], 'GSM8475059': [1.0, 59.0, 0.0], 'GSM8475060': [1.0, 40.0, 1.0], 'GSM8475061': [1.0, 61.0, 1.0], 'GSM8475062': [1.0, 47.0, 0.0], 'GSM8475063': [1

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/COVID-19/GSE275334/GSE275334_family.soft.gz
Matrix file: ../../input/GEO/COVID-19/GSE275334/GSE275334_series_matrix.txt.gz
Found the matrix table marker at line 65
Gene data shape: (635, 47)
First 20 gene/probe identifiers:
['ACACA', 'ACADVL', 'ACAT2', 'ACOT1/2', 'ACSL3', 'ACSL4', 'ACSL6', 'ADORA2A', 'ADORA2B', 'AHR', 'AIFM1', 'AK4', 'AKT1', 'AKT2', 'AKT3', 'ALDH1A1', 'ALDH1B1', 'ALDOA', 'ALOX5', 'ANAPC4']


### Step 4: Gene Identifier Review

In [5]:
# Based on the first 20 identifiers shown, these appear to be human gene symbols.
# The identifiers are in the format of human gene symbols like ACACA, ACADVL, etc.
# There is a mix of standard gene symbols and some combined identifiers (e.g., ACOT1/2)
# but overall these are human gene symbols and don't require mapping.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols and prepare for linking
try:
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Save the gene data
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene data saved to {out_gene_data_file}")
    
    # Attempt to link clinical and gene data
    if 'trait_row' in locals() and trait_row is not None:
        # Load clinical data from the previous step
        try:
            clinical_features = pd.read_csv(out_clinical_data_file)
            print(f"Loaded clinical data shape: {clinical_features.shape}")
            
            # Convert clinical_features to the correct format for linking
            clinical_features.set_index(clinical_features.columns[0], inplace=True)
            clinical_features = clinical_features.T
            clinical_features.columns = [trait, 'Age', 'Gender']
            
            print("Clinical features columns after transformation:", clinical_features.columns.tolist())
            
            # Link the clinical and genetic data
            linked_data = pd.concat([clinical_features, gene_data.T], axis=1)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate data quality and save cohort info
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=True,
                    is_biased=is_biased,
                    df=linked_data,
                    note="Successfully processed gene expression data for COVID-19."
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=True,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        except Exception as e:
            print(f"Error loading or processing clinical data: {e}")
            # Try to create a minimal response in case of error
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=False,
                is_biased=True,
                df=pd.DataFrame(),
                note=f"Error processing clinical data: {str(e)}"
            )
    else:
        # Cannot proceed with linking if trait data is missing
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=False,
            is_biased=True,
            df=pd.DataFrame(),
            note="Cannot link data because trait information is not available."
        )
except Exception as e:
    print(f"Error in data processing: {e}")
    
    # Log the error and mark the dataset as unusable
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False,
        is_biased=True,
        df=pd.DataFrame(),
        note=f"Error during normalization or linking: {str(e)}"
    )

Gene data saved to ../../output/preprocess/COVID-19/gene_data/GSE275334.csv
Loaded clinical data shape: (3, 47)
Clinical features columns after transformation: ['COVID-19', 'Age', 'Gender']
Initial linked data shape: (47, 638)
Linked data shape after handling missing values: (46, 638)
For the feature 'COVID-19', the least common label is '1.0' with 15 occurrences. This represents 32.61% of the dataset.
The distribution of the feature 'COVID-19' in this dataset is fine.

Quartiles for 'Age':
  25%: 34.75
  50% (Median): 43.5
  75%: 50.75
Min: 19.0
Max: 63.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 12 occurrences. This represents 26.09% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Linked data saved to ../../output/preprocess/COVID-19/GSE275334.csv
